In [2]:
import requests
from bs4 import BeautifulSoup 
from urllib.request import urlopen
from selenium import webdriver
import pandas as pd
import time

# 순서
- 셀레니움으로 원하는 페이지 열기
1. 2017년 버튼 누르기
2. 원하는 시도 버튼 누르기
3. 원하는 시도의 상세주소정보 누르기
4. 페이지 내의 정보 크롤링하기
5. 다음 페이지 클릭하고 페이지 끝까지 4번 반복
6. 마지막페이지에서 3~5 반복


- 지역 하나 다 끝냈으면 다른 지역 선택해서 반복

In [3]:
# chromedriver 구동, url 설정
driver = webdriver.Chrome('./chromedriver')
url ='https://apt2.me/apt/high.jsp#read/page=1&perPage=20' 
driver.get(url)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
#2017년 선택
driver.find_element_by_xpath('//*[@id="Cmb_year"]/option[2]').click()

In [ ]:
df = pd.DataFrame(columns=["학교명주소","서울대 계수시정시","KAIST 계수시정시","서울대 + KAIST"])

In [ ]:
sido = ['제주특별자치도']

In [ ]:
for si in sido:
    driver.find_element_by_xpath('/html/body/a').click() 
    time.sleep(0.5)
    #서울부터 지역 선택
    driver.find_element_by_link_text(si).click()
    time.sleep(0.3)
    gungu_len = len(driver.find_element_by_xpath('//*[@id="area"]').text.split(' '))
    time.sleep(0.5)

    for i in range(1,gungu_len+1):
        if si == '인천광역시' and i == 4:
            continue
        elif si == '경기도' and i ==35:
            continue
        elif si == '강원도' and i == 17:
            continue
        elif si == '충청북도' and i==2:
            continue
        elif si =='충청남도' and i==16:
            continue
        elif si =='전라북도' and i ==6:
            continue
        elif si =='전라북도' and i ==11:
            continue
        elif si =='전라북도' and i ==16:
            continue
        elif si =='전라남도' and i ==1:
            continue
        elif si =='전라남도' and i ==3:
            continue 
        elif si =='전라남도' and i ==5:
            continue 
        elif si =='전라남도' and i ==20:
            continue   
        elif si =='경상북도' and i ==12:
            continue   
        elif si =='경상북도' and i ==17:
            continue
        elif si =='경상북도' and i ==18:
            continue  
        elif si =='경상북도' and i ==19:
            continue 
        elif si =='경상북도' and i ==21:
            continue
        elif si =='경상남도' and i ==3:
            continue
        elif si =='경상남도' and i ==8:
            continue
        elif si =='경상남도' and i ==10:
            continue
        #지역 선택
        driver.find_element_by_link_text(si).click()
        time.sleep(0.5)
        #첫번째 구부터 선택
        driver.find_element_by_xpath('/html/body/div[3]/a['+str(i)+']').click()
        time.sleep(0.5)
        #페이지 수
        page_num = int(driver.find_element_by_css_selector('#grid > div.tui-pagination.tui-grid-pagination').text.replace("firstprev","").replace("nextlast",""))

        # df 만들기
        html = driver.page_source
        table_df_list =pd.read_html(html)

        table_df_list[3].rename({0:"학교명주소",
                   1:"서울대 계수시정시	",
                   2:"KAIST 계수시정시",
                   3:"서울대 + KAIST"
                  },axis=1,inplace=True)

        df = pd.concat([ table_df_list[3], df])
        driver.find_element_by_xpath('/html/body/a').click()
        time.sleep(0.5)

In [ ]:
df = df.drop_duplicates(keep='first')

In [ ]:
df.info()

In [ ]:
df.to_csv('../Data/학교_Data/원본_고등학교 입시실적 크롤링.csv')

In [ ]:
df

# 나중에 전처리할 코드

In [ ]:
table_df_list[3]['학교명'] = [i.split('중학교')[0]+'중학교' for i in table_df_list[3][0]]
table_df_list[3]['주소'] = [i.split('중학교')[1].split(' ')[0]+" "+i.split('중학교')[1].split(' ')[1] for i in table_df_list[3][0]]

table_df_list[3]['총인원'] = [i.split('명')[0].split(' ')[0] for i in table_df_list[3][1]]
table_df_list[3]['남학생'] = [i.split('명')[1].split(' ')[0] for i in table_df_list[3][1]]
table_df_list[3]['여학생'] = [i.split('명')[2].split(' ')[0] for i in table_df_list[3][1]] 

table_df_list[3]['과학고'] = [i.split('명')[0].split(' ')[0] for i in table_df_list[3][2]]
table_df_list[3]['외고/국제고'] = [i.split('명')[1].split(' ')[0] for i in table_df_list[3][2]]
table_df_list[3]['자사고'] = [i.split('명')[2].split(' ')[0] for i in table_df_list[3][2]]
table_df_list[3]['기타계(외국,영재,유학,대안)'] = [i.split('명')[3].split(' ')[0] for i in table_df_list[3][2]]

table_df_list[3]['특목고계'] = [i.split('명')[0].split(' ')[0] for i in table_df_list[3][3]]
table_df_list[3]['특목고비율(%)'] = [i.split('명')[1].split(' ')[0] for i in table_df_list[3][3]]

df = table_df_list[3].drop([0,1,2,3],axis=1)

In [ ]:
for _ in range(iter_num):
    driver.find_elements_by_link_text('next')[0].click()
    time.sleep(0.5) #이거 안넣으면 다 긁기도 전에 다음페이지 클릭해버려서 에러남
    #페이지 html
    html = driver.page_source
    table_df_list =pd.read_html(html)

    # dataframe 정리하고 합치기
    table_df_list[3]['학교명'] = [i.split('중학교')[0]+'중학교' for i in table_df_list[3][0]]
    table_df_list[3]['주소'] = [i.split('중학교')[1].split(' ')[0]+" "+i.split('중학교')[1].split(' ')[1] for i in table_df_list[3][0]]

    table_df_list[3]['총인원'] = [i.split('명')[0].split(' ')[0] for i in table_df_list[3][1]]
    table_df_list[3]['남학생'] = [i.split('명')[1].split(' ')[0] for i in table_df_list[3][1]]
    table_df_list[3]['여학생'] = [i.split('명')[2].split(' ')[0] for i in table_df_list[3][1]] 

    table_df_list[3]['과학고'] = [i.split('명')[0].split(' ')[0] for i in table_df_list[3][2]]
    table_df_list[3]['외고/국제고'] = [i.split('명')[1].split(' ')[0] for i in table_df_list[3][2]]
    table_df_list[3]['자사고'] = [i.split('명')[2].split(' ')[0] for i in table_df_list[3][2]]
    table_df_list[3]['기타계(외국,영재,유학,대안)'] = [i.split('명')[3].split(' ')[0] for i in table_df_list[3][2]]

    table_df_list[3]['특목고계'] = [i.split('명')[0].split(' ')[0] for i in table_df_list[3][3]]
    table_df_list[3]['특목고비율(%)'] = [i.split('명')[1].split(' ')[0] for i in table_df_list[3][3]]

    df = table_df_list[3].drop([0,1,2,3],axis=1)
    df_final = pd.concat([df_final,df])